In [1]:
import os
os.environ["SPARK_HOME"] = "/Applications/spark"
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_PYTHON"] = "python"

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark-ml') \
                .getOrCreate()

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/27 19:32:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import pyspark.sql.functions as F
from pyspark.sql.types import (
    StructField, 
    StructType, 
    IntegerType, 
    FloatType, 
    StringType,
)
from pyspark.sql import DataFrame
from pyspark.sql.functions import udf

In [4]:
data = spark.read.parquet("../dataset/month_partition_online_shoppers_intention", header= True, inferSchema = True)

data.show()

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+-----------+-----------+----------+----------------+-------+------+-----------+-----------------+-------+-------------+-------------------+-----+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|  ExitRates| PageValues|SpecialDay|OperatingSystems|Browser|Region|TrafficType|      VisitorType|Weekend|made_purchase|ingestion_timestamp|Month|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+-----------+-----------+----------+----------------+-------+------+-----------+-----------------+-------+-------------+-------------------+-----+
|             1|                   39.2|            2|                 120.8|             7|                   80.5|        0.0|       0.01|        0.0|       0.0|               

##### 1. Create TotalDuration by summing all duration columns

In [5]:
data = data.withColumn(
    "TotalDuration", 
    F.col('Administrative_Duration') + F.col('Informational_Duration') + F.col('ProductRelated_Duration')
)

cols = ["TotalDuration", "Administrative_Duration", "Informational_Duration", "ProductRelated_Duration"]
data.select(*cols).show()

+------------------+-----------------------+----------------------+-----------------------+
|     TotalDuration|Administrative_Duration|Informational_Duration|ProductRelated_Duration|
+------------------+-----------------------+----------------------+-----------------------+
|             240.5|                   39.2|                 120.8|                   80.5|
|1811.5066669999999|                   89.6|                   0.0|            1721.906667|
|       856.5766667|                  204.2|                   0.0|            652.3766667|
|       710.0666667|                    0.0|                   0.0|            710.0666667|
|       968.6924242|                    0.0|                   0.0|            968.6924242|
|       3812.879013|            1013.056909|                 102.8|            2697.022104|
|436.17619049999996|                   86.6|                   0.0|            349.5761905|
|      1932.4289997|                 545.65|           204.3466667|            1

##### 2.	Bucket PageValues into Low / Medium / High.

In [6]:
data.select('PageValues').describe().show()

+-------+-----------------+
|summary|       PageValues|
+-------+-----------------+
|  count|            12330|
|   mean|5.889257862693592|
| stddev|18.56843660780653|
|    min|              0.0|
|    max|      361.7637419|
+-------+-----------------+



In [7]:
(data.select(F.percentile(col = 'PageValues', 
                        percentage = [0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90], frequency = 1)
                        .alias("percentileValues")).collect()[0][0])

# We see almost around 70% values are null 

# Bucket -> Low  where value == 0.0
# Bucket -> Medium where value > 0 and value <= 3.07
# Bucket -> hight where value > 3.07


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.060078087400001, 18.855502398000006]

In [8]:
data = (data.withColumn("PageValuesBucket", 
                 F.when(F.col('PageValues') == 0.0, F.lit('low'))
                .when(((F.col('PageValues') > 0) & (F.col('PageValues') <= 3.07)), F.lit('Medium'))
                .otherwise(F.lit("High"))))


data.select(*['PageValues', 'PageValuesBucket']).show()


# Counts
data.groupBy("PageValuesBucket").count().show()

+-----------+----------------+
| PageValues|PageValuesBucket|
+-----------+----------------+
|        0.0|             low|
|204.0079491|            High|
|        0.0|             low|
|72.52283848|            High|
|106.2525169|            High|
|1.798669862|          Medium|
|60.43737784|            High|
|19.71529542|            High|
|        0.0|             low|
|        0.0|             low|
|        0.0|             low|
|        0.0|             low|
| 77.4579855|            High|
|26.13034871|            High|
|        0.0|             low|
|44.85638692|            High|
|        0.0|             low|
|        0.0|             low|
|20.94544525|            High|
|10.72117234|            High|
+-----------+----------------+
only showing top 20 rows

+----------------+-----+
|PageValuesBucket|count|
+----------------+-----+
|            High| 2464|
|             low| 9600|
|          Medium|  266|
+----------------+-----+



##### 3. Encode Month into a numeric month index.

In [9]:
data.select(F.from_unixtime(F.unix_timestamp(F.col("Month"), "MMM"), 'MM')).show()

+---------------------------------------------+
|from_unixtime(unix_timestamp(Month, MMM), MM)|
+---------------------------------------------+
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                           11|
|                                       

In [15]:
data = data.withColumn('Month_Num', 
        F.from_unixtime(F.unix_timestamp(F.col("Month"), "MMM"), 'MM'))


data.select(F.col("Month"), F.col("Month_Num")).show()

+-----+---------+
|Month|Month_Num|
+-----+---------+
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
|  nov|       11|
+-----+---------+
only showing top 20 rows



##### 4. Normalize BounceRates using min‑max scaling

In [36]:
maxVal, minVal = data.select(F.max('BounceRates'), F.min('BounceRates')).collect()[0][0], data.select(F.max('BounceRates'), F.min('BounceRates')).collect()[0][1]

print(f"maxVal : {maxVal} and minVal : {minVal}")

maxVal : 0.2 and minVal : 0.0


In [39]:
@udf(returnType = FloatType())
def min_max_sampling(val):
    scaled_val =(val - minVal) / (maxVal - minVal)
    return scaled_val


data = data.withColumn('scaledBounceRates', min_max_sampling(F.col('BounceRates')))
cols = ["BounceRates", "scaledBounceRates"]
data.select(*cols).show()

+-----------+-----------------+
|BounceRates|scaledBounceRates|
+-----------+-----------------+
|        0.0|              0.0|
|        0.0|              0.0|
|0.012121212|       0.06060606|
|        0.0|              0.0|
|        0.0|              0.0|
|        0.0|              0.0|
|        0.0|              0.0|
| 6.84932E-4|       0.00342466|
|        0.0|              0.0|
|        0.0|              0.0|
|        0.0|              0.0|
|        0.0|              0.0|
|        0.0|              0.0|
|0.015384615|       0.07692307|
|0.003773585|      0.018867925|
|        0.0|              0.0|
|        0.2|              1.0|
|        0.0|              0.0|
|       0.04|              0.2|
|0.021212121|        0.1060606|
+-----------+-----------------+
only showing top 20 rows



##### 5. Create interaction feature: ProductRelated * PageValues.

In [41]:
@udf(returnType = FloatType())
def interactionfeature(productrel, pageVal):
    return productrel * pageVal


data = data.withColumn("InteractionFeat", interactionfeature(F.col('ProductRelated'), F.col('PageValues')))

data.select(["InteractionFeat", "ProductRelated", "PageValues"]).show()
    

+---------------+--------------+-----------+
|InteractionFeat|ProductRelated| PageValues|
+---------------+--------------+-----------+
|            0.0|             7|        0.0|
|      11628.453|            57|204.0079491|
|            0.0|            31|        0.0|
|       942.7969|            13|72.52283848|
|      2550.0603|            24|106.2525169|
|      138.49757|            77|1.798669862|
|       846.1233|            14|60.43737784|
|      1360.3553|            69|19.71529542|
|            0.0|            16|        0.0|
|            0.0|            12|        0.0|
|            0.0|            11|        0.0|
|            0.0|             0|        0.0|
|      1936.4496|            25| 77.4579855|
|      679.38904|            26|26.13034871|
|            0.0|            48|        0.0|
|      1211.1224|            27|44.85638692|
|            0.0|             0|        0.0|
|            0.0|            10|        0.0|
|      125.67267|             6|20.94544525|
|      407

##### 6. StringIndexer Visitor Type Column

In [48]:
data.select('VisitorType').distinct().show()

+-----------------+
|      VisitorType|
+-----------------+
|      New_Visitor|
|            Other|
|Returning_Visitor|
+-----------------+



In [47]:
from pyspark.ml.feature import StringIndexer
strIndexer = StringIndexer(inputCol = 'VisitorType', outputCol = 'VisitorType_indexer')
strIndexer.setHandleInvalid('error')

# fit strIndexer to the dataset
model = strIndexer.fit(data)

model.transform(data).select(['VisitorType', 'VisitorType_indexer']).show()

+-----------------+-------------------+
|      VisitorType|VisitorType_indexer|
+-----------------+-------------------+
|      New_Visitor|                1.0|
|Returning_Visitor|                0.0|
|Returning_Visitor|                0.0|
|Returning_Visitor|                0.0|
|Returning_Visitor|                0.0|
|Returning_Visitor|                0.0|
|      New_Visitor|                1.0|
|Returning_Visitor|                0.0|
|      New_Visitor|                1.0|
|Returning_Visitor|                0.0|
|Returning_Visitor|                0.0|
|      New_Visitor|                1.0|
|      New_Visitor|                1.0|
|Returning_Visitor|                0.0|
|Returning_Visitor|                0.0|
|      New_Visitor|                1.0|
|Returning_Visitor|                0.0|
|      New_Visitor|                1.0|
|Returning_Visitor|                0.0|
|Returning_Visitor|                0.0|
+-----------------+-------------------+
only showing top 20 rows



##### 7. One‑hot encode Month.

In [55]:
# Change the data type of the Month_Num column
data = data.withColumn('Month_Num', F.col('Month_Num').cast('float'))

data.printSchema()

root
 |-- Administrative: integer (nullable = true)
 |-- Administrative_Duration: double (nullable = true)
 |-- Informational: integer (nullable = true)
 |-- Informational_Duration: double (nullable = true)
 |-- ProductRelated: integer (nullable = true)
 |-- ProductRelated_Duration: double (nullable = true)
 |-- BounceRates: double (nullable = true)
 |-- ExitRates: double (nullable = true)
 |-- PageValues: double (nullable = true)
 |-- SpecialDay: double (nullable = true)
 |-- OperatingSystems: integer (nullable = true)
 |-- Browser: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- TrafficType: integer (nullable = true)
 |-- VisitorType: string (nullable = true)
 |-- Weekend: boolean (nullable = true)
 |-- made_purchase: boolean (nullable = true)
 |-- ingestion_timestamp: date (nullable = true)
 |-- Month: string (nullable = true)
 |-- TotalDuration: double (nullable = true)
 |-- PageValuesBucket: string (nullable = false)
 |-- Month_Num: float (nullable = true)
 |

In [58]:
data.select('Month_Num').distinct().show()

26/01/27 22:38:18 ERROR Executor: Exception in task 8.0 in stage 71.0 (TID 258)
org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse 'june' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54)
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError$(ExecutionErrors.scala:48)
	at org.apache.spark.sql.errors.ExecutionErrors$.failToParseDateTimeInNewParserError(ExecutionErrors.scala:218)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:142)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$an

Py4JJavaError: An error occurred while calling o704.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 71.0 failed 1 times, most recent failure: Lost task 8.0 in stage 71.0 (TID 258) (mac.attlocal.net executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse 'june' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54)
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError$(ExecutionErrors.scala:48)
	at org.apache.spark.sql.errors.ExecutionErrors$.failToParseDateTimeInNewParserError(ExecutionErrors.scala:218)
	at org.apache.spa

In [56]:
from pyspark.ml.feature import OneHotEncoder
onehotencoder = OneHotEncoder(inputCol = 'Month_Num', outputCol = 'Month_OneHot')
model = onehotencoder.fit(data)

model.transform(data).select(['Month_Num', 'Month_OneHot']).show()

26/01/27 22:37:18 ERROR Executor: Exception in task 8.0 in stage 69.0 (TID 248)
org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse 'june' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54)
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError$(ExecutionErrors.scala:48)
	at org.apache.spark.sql.errors.ExecutionErrors$.failToParseDateTimeInNewParserError(ExecutionErrors.scala:218)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:142)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$an

Py4JJavaError: An error occurred while calling o667.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 69.0 failed 1 times, most recent failure: Lost task 8.0 in stage 69.0 (TID 248) (mac.attlocal.net executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse 'june' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54)
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError$(ExecutionErrors.scala:48)
	at org.apache.spark.sql.errors.ExecutionErrors$.failToParseDateTimeInNewParserError(ExecutionErrors.scala:218)
	at org.apache.spark.sql.